# Projeto Final - Sistemas de Recuperação da Informação - Parte 1

### Julio Bruce da Motta Azevedo

## INDEXAÇÃO

## Biblioteca Whoosh

In [1]:
# Importando as bibliotecas necessárias (Whoosh, nossa 'search-engine' de estudo; OS, para interface com o sistema operacional;
# e IPython.core.display, para mostrar os resultados).

import os
from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh.analysis import CharsetFilter, StemmingAnalyzer
from whoosh import fields
from whoosh.support.charset import accent_map
from IPython.core.display import display, HTML

In [2]:
# Definição do 'schema' (campos dos documentos em um índice), bem como criação do diretório local para trabalhar o corpus.

analizer = StemmingAnalyzer() | CharsetFilter(accent_map)
schema = Schema(content=TEXT(stored=True, analyzer=analizer), tid=TEXT(stored=True))
if os.path.exists('dir_index'):
    ix = open_dir('dir_index')
else:
    os.mkdir('dir_index')
    ix = create_in('dir_index', schema)

In [3]:
# Importando o corpus escolhido (Reuters).

import nltk
from nltk.corpus import reuters

In [4]:
# Criação do 'texto-puro' do corpus Reuters.

textos = [reuters.raw(fid) for fid in reuters.fileids()]

In [5]:
# Após a definição do 'schema', podemos modificá-lo ou adicionar campos (documentos), com métodos do objeto 'writer'.

writer = ix.writer()
for tid in reuters.fileids():
    writer.add_document(content=reuters.raw(tid), tid=tid)
writer.commit()

In [6]:
# 'QueryParser' converte a string inserida pela consulta do usuário em uma consulta de objetos.

from whoosh.qparser import QueryParser

In [7]:
# Apresentação (com destaque via 'hit.highlights') dos TOP10 documentos que podem atender à consulta, com respectivos trechos.
# Por padrão, o Whoosh ordena os resultados da consulta ('high-score').
# Pode-se também, caso se queira, usar o argumento 'sortedby' para usar outro critério de ordenação.

qw = "politics and economics of canada"
qp = QueryParser("content", ix.schema)
query = qp.parse(qw)
with ix.searcher() as searcher:
    results = searcher.search(query)
    for hit in results:
        tid = hit.fields()['tid']
        display(HTML(f'<b>{tid}: </b'+ hit.highlights("content")))